In [ ]:
import pandas as pd
from scipy.stats import norm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
import os
import numpy as np
import seaborn as sns

In [ ]:
# read the Ginkgo CNV calls with empirical p-value generated from the last step
cnv_df = pd.read_pickle("STEP_7_out_CNV.pkl")

In [ ]:
true_cnv_df.head()

In [ ]:
true_cnv_df.shape

In [ ]:
# select p-value < 0.05 as strong calls
strong_cnv = cnv_df[cnv_df["pvalue"]<0.05]
strong_ratio_list = strong_cnv.median_log2.to_list()
print(len(strong_ratio_list))

In [ ]:
# read the permutated non-CNV calls with empirical p-value generated from the last step
nonCNV_df = pd.read_pickle("STEP_7_out_nonCNV.pkl"")

In [ ]:
# as the number of non-CNV is much larger than the CNV, we recommend use the first K non-CNVs to build the model
# K is a little larger than the number of strong calls, like twice of that.
# Users should define K here
K = 
nonCNV_df = nonCNV_df.iloc[:K]

In [ ]:
nonCNV_df.head()

In [ ]:
nonCNV_ratio_list = nonCNV_df.median_log2.dropna().to_list()

In [ ]:
(mu_cnv, sigma_cnv) = norm.fit(strong_ratio_list)
(mu_non, sigma_non) = norm.fit(nonCNV_ratio_list)

In [ ]:
print(mu_cnv, sigma_cnv)
print(mu_non, sigma_non) 

In [ ]:
#new color scheme
bins = np.linspace(0, 8, 80)

n2, bin2, patches2 = plt.hist(nonCNV_ratio_list, bins, alpha=0.5, density=True, label='non-CNV permutations')
n1, bin1, patches1 = plt.hist(strong_ratio_list, bins, alpha=0.5, density=True, label='strong CNVs (p-value<0.05)')

plt.legend(loc='upper right')

y1 = norm.pdf(bin1, mu_cnv, sigma_cnv)
y2 = norm.pdf(bin2, mu_non, sigma_non)
plt.plot(bin1, y1, '--')
plt.plot(bin2, y2, '--')
plt.xlabel("median log2 ratio")
plt.ylabel("density")
plt.show()

In [ ]:
cnv_noNA = true_cnv_df[~true_cnv_df["median_log2"].isna()].copy()

In [ ]:
strong_arr = np.array(strong_ratio_list)
non_arr = np.array(nonCNV_ratio_list)
all_data = np.concatenate((strong_arr, non_arr), axis=0)
all_data = all_data.reshape(-1, 1)

In [ ]:
gmm = GaussianMixture(n_components=2, max_iter=1000, covariance_type='full').fit(all_data)

In [ ]:
posterior = gmm.predict_proba(np.array(cnv_noNA.median_log2.to_list()).reshape(-1,1))

In [ ]:
cnv_prob = [x[0] for x in posterior]

In [ ]:
cnv_noNA["posterior_prob"] = cnv_prob

In [ ]:
cnv_noNA.posterior_prob.plot.hist(bins=100)
plt.title("posterior probability distribution")

In [ ]:
cnv_noNA[cnv_noNA["posterior_prob"]> 0.6].shape

In [ ]:
cnv_noNA[cnv_noNA["posterior_prob"]> 0.99].shape

In [ ]:
cnv_noNA.head()

In [ ]:
out_cnv = pd.merge(true_cnv_df, cnv_noNA, how="inner", on=["chrom", "start", "end", "bam", "copy_number","barcode","size","median_log2","pvalue"])

In [ ]:
out_cnv = out_cnv.query("posterior_prob > 0.99").copy()

In [ ]:
out_cnv.reset_index(drop=True, inplace=True)

In [ ]:
out_cnv.to_pickle("STEP_8_out.pkl")